# Data import

data를 가져오는 부분

In [1]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [2]:
quantifi_data = data_array[1:,:]
print(quantifi_data[0])

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

['1' '0' '3' 'Braund, Mr. Owen Harris' 'male' '22' '1' '0' 'A/5 21171'
 '7.25' '' 'S']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [3]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


#### 성별과 동승자수로만 분류해보자
#### x data size는 [datanum, 2(성별, 동승자수)]
#### y data size는 [datanum, 1]이다

In [4]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,4], quantifi_data[:,6]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,3], quantifi_data1[:,5]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([2,2]), name='weight1')
b1 = tf.Variable(tf.random_normal([2]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2,1]), name='weight2')
b2 = tf.Variable(tf.random_normal([1]), name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 1.0529069 [array([[-1.2975113,  1.2943718],
       [ 0.8073871,  1.297885 ]], dtype=float32), array([[-1.2716902],
       [ 0.2533999]], dtype=float32)]
100 0.4994148 [array([[-3.8668544,  5.928792 ],
       [ 1.324076 ,  3.7587001]], dtype=float32), array([[-2.4974015],
       [ 2.0325716]], dtype=float32)]
200 0.4963459 [array([[-3.7103672,  6.1356826],
       [ 1.2721301,  5.720165 ]], dtype=float32), array([[-3.181655 ],
       [ 2.1084943]], dtype=float32)]
300 0.4951443 [array([[-3.2431192,  6.333985 ],
       [ 1.2139745,  6.376119 ]], dtype=float32), array([[-3.6268172],
       [ 2.202213 ]], dtype=float32)]
400 0.49473014 [array([[-2.9982421,  6.5412908],
       [ 1.1530915,  6.475088 ]], dtype=float32), array([[-3.98394  ],
       [ 2.2440057]], dtype=float32)]
500 0.49457633 [array([[-2.813277 ,  6.688117 ],
       [ 1.0808008,  6.3214116]], dtype=float32), array([[-4.225716],
       [ 2.250262]], dtype=float32)]
600 0.4944892 [array([[-2.6500306,  6.7699866],
       [ 1.0

5200 0.49412525 [array([[-1.4898468 ,  4.1451616 ],
       [ 0.65039074,  3.13875   ]], dtype=float32), array([[-6.394606],
       [ 2.498758]], dtype=float32)]
5300 0.49438745 [array([[-1.5007786,  4.112666 ],
       [ 0.6297097,  3.1353366]], dtype=float32), array([[-6.4138665],
       [ 2.5189195]], dtype=float32)]
5400 0.4941255 [array([[-1.4873925,  4.10972  ],
       [ 0.6495442,  3.1052027]], dtype=float32), array([[-6.403302 ],
       [ 2.5177317]], dtype=float32)]
5500 0.4941253 [array([[-1.4789791,  4.058365 ],
       [ 0.645351 ,  3.0808728]], dtype=float32), array([[-6.428779 ],
       [ 2.5318885]], dtype=float32)]
5600 0.49412546 [array([[-1.4877167 ,  4.097447  ],
       [ 0.65059906,  3.0721433 ]], dtype=float32), array([[-6.4096274],
       [ 2.5363824]], dtype=float32)]
5700 0.4941245 [array([[-1.4782418,  4.0270987],
       [ 0.6453625,  3.0498643]], dtype=float32), array([[-6.430492 ],
       [ 2.5512712]], dtype=float32)]
5800 0.4941388 [array([[-1.4667234,  3.9763

test_prediction의 값을 0과 1로 정규화 시켜준다

In [5]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]

예측값들을 csv파일로 저장한다

In [6]:
f = open('prediction.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()